## -1. Instructions:
* Run the cells below sequentially
* Click a cell to edit
* ** Shift-enter ** to run a cell

## 0. Import useful tools

In [ ]:
# Be patient, this may take a little while
# If you see "In [*]" to the left of this cell, it means it is busy computing, just wait a bit longer
import os, imars3d, numpy as np

In [ ]:
# Be patient, this may take a little while too
# %matplotlib notebook
%matplotlib inline
from matplotlib import pyplot as plt

## 1. Define data paths
** Modify these paths to suit your need **

In [ ]:
# use your experiment IPTS number
iptsdir = "/HFIR/CG1DImaging/IPTS-17413/"

In [ ]:
# path to the directory with ct, ob, and df data files or subdirs
datadir = os.path.join(iptsdir,"raw/")

In [ ]:
# temporary working directory. 
# !!! should be /SNSlocal2/<yourusername>/<experimentname> !!!
workdir = "/SNSlocal2/lj7/crack3-try1"
# !rm -rf {workdir}
!mkdir -p {workdir}

In [ ]:
# path to save output results (reconstructed slices)
outdir = os.path.join(iptsdir, "shared/processed_data/crack3-try1")

## 2. Create CT data object

In [ ]:
# check CT scan data files.
# You will need to make sure the
# "CT_identifier" string you choose is correct.
# In this example, the CT_identifier is "Cracked_3_CT"
!ls {datadir}/ct_scans/Cracked_3/*Cracked_3_CT* | sed -n "1,6p"

In [ ]:
# check OB data files.
# You will need to make sure the
# "ob_identifier" string you choose is correct.
# In this example, the ob_identifier is "Cracked_3"
!ls {datadir}/ob/*Cracked_3*

In [ ]:
# check DF data files
# You will need to make sure the
# "df_identifier" string you choose is correct.
# In this example, the df_identifier is "Cracked_2_DF"
!ls {datadir}/df/*Cracked_2_DF*

In [ ]:
# import the CT class
from imars3d.CT import CT

In [ ]:
# ask for help
CT?

In [ ]:
# Once the filename patterns are confirmed, 
# you can create the ct object.
ct = CT(datadir, 
        CT_subdir="ct_scans/Cracked_3", CT_identifier="Cracked_3_CT", 
        workdir=workdir, outdir=outdir,
        ob_identifier="Cracked_3", df_identifier="Cracked_2_DF"
       )

## 3. Find region of interests

In [ ]:
# compute an average image in order to obtain ROI
# it may take a minute
ave = ct.estimateAverage(ct.ct_series)

In [ ]:
# plot the average image. it may take a few seconds
plt.pcolormesh(ave, cmap="gray")
plt.colorbar()

** For this sample, we can define the ROI as xmin=750, ymin=0, xmax=1300, ymax=2047 **

## 4. Run reconstruction

In [ ]:
# Now run the reconstruction. 
# Make sure to modify the crop_window and use the numbers obtained from above
# This will take 10-60 minutes.
ct.recon(crop_window=(750,0,1300,2047))

** Check output **

In [ ]:
# recon output
!ls -l {outdir} |head -n 10

In [ ]:
# temporary results
!ls -l {workdir}

In [ ]:
# sinograms
!ls {workdir}/sinogram | head -n 10

** Visualization **

ImageJ. To start, open a terminal and type
```
$ /HFIR/CG1DImaging/shared/Fiji.app/ImageJ-linux64
```